- **Name:** 06_dataframe_basic-operations
- **Author:** Shamas Imran
- **Desciption:** Performing basic DataFrame operations
- **Date:** 19-Aug-2025
<!--
REVISION HISTORY
Version          Date        Author           Desciption
01           19-Aug-2025   Shamas Imran       Selected columns and filtered rows  
                                              Added new derived columns
                                              Rename/Drp column for Dataframe
                                              Apply sort/orderby  
                                              Demonstrated basic aggregations  
-->

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DatapurProgram").getOrCreate()

In [0]:
rootPath = "/Volumes/datapurcatalog/default/datapurvolume/"
folderpath = rootPath + "student.csv"
df_student = spark.read.option("header", "true").csv(folderpath)
print("CSV file contents:")
display(df_student)

In [0]:
df_studentsubset = df_student.select(df_student.StudentID, df_student.StudentName, df_student.StudentAge.alias("Age"))
display(df_studentsubset)

In [0]:
df_studentsubset.filter(df_studentsubset.Age > 30).show() # Filter rows

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

df_studentsubset_renamed = df_studentsubset.withColumnRenamed("Age", "SolarAge")

df_studentFinal = (
    df_studentsubset_renamed
    .withColumn(
        "LunarAge",
        (
            (
                (df_studentsubset_renamed.SolarAge * 365) +
                (df_studentsubset_renamed.SolarAge * 11)
            ) / 365
        ).cast("int")
    )
)

df_studentFinal = df_studentFinal.withColumn("row_id", monotonically_increasing_id())

display(df_studentFinal)

In [0]:
# Sort by Age (ascending - default)
df_studentFinal = df_studentFinal.orderBy("StudentID").show() # or desc()
df_studentFinal = df_studentFinal.orderBy("StudentID").desc().show()

# Sort by StudentID and
df_studentFinal = df_studentFinal.orderBy("StudentID", "LunarAge").desc().show()

In [0]:
df_studentFinal = df_studentFinal.drop("SolarAge")
display(df_studentFinal)